
----
# Cuaderno 6 - Regresión Logística y Evaluación de Clasificadores
## Ariel Palazzesi - 2026
----

En este cuaderno vamos a construir un modelo de **regresión logística** para resolver un problema de clasificación binaria.

Trabajaremos nuevamente con el conjunto de datos del Titanic y buscaremos predecir si una persona **sobrevivió** o no al hundimiento del barco, en función de ciertas características personales.

Además de entrenar el modelo, vamos a evaluarlo utilizando métricas específicas para clasificación, como la **precisión (accuracy)**, el **recall**, el **F1-score** y la **matriz de confusión**.

El objetivo no es solo construir el modelo, sino **entender cómo se comporta, cómo se equivoca y qué tan confiable es**.


## Un poco de teoría

Antes de comenzar, repasemos algunos conceptos.

La **regresión logística** es un algoritmo de **aprendizaje supervisado** que se utiliza en problemas de **clasificación**, especialmente cuando la variable objetivo tiene dos posibles valores (clasificación binaria), como por ejemplo *sí / no*, *0 / 1* o *sobrevive / no sobrevive*. A diferencia de los modelos de regresión, su objetivo no es predecir un valor numérico continuo, sino **asignar una clase** a cada observación.

Este modelo funciona estimando la **probabilidad** de que una observación pertenezca a una determinada clase. Internamente utiliza una función logística (o sigmoide) que transforma una combinación lineal de las variables de entrada en un valor entre 0 y 1. A partir de esa probabilidad, el modelo decide la clase final usando un umbral (habitualmente 0.5). Por su simplicidad, rapidez y facilidad de interpretación, la regresión logística suele utilizarse como **modelo base** en problemas de clasificación.

En scikit-learn, la regresión logística se implementa mediante la clase `LogisticRegression`, que sigue la misma interfaz que otros modelos: primero se entrena con `fit()` y luego se generan predicciones con `predict()`.

---

### Métricas de evaluación en clasificación

A diferencia de los problemas de regresión, donde se mide el error numérico de una predicción, en clasificación interesa evaluar **qué tan bien el modelo asigna las clases correctas**. Para ello se utilizan métricas específicas.

* La **precisión (accuracy)** mide la proporción de predicciones correctas sobre el total de observaciones. Es fácil de interpretar, pero puede ser engañosa cuando las clases están desbalanceadas. Por ejemplo, si una clase es muy dominante, un modelo puede tener alta precisión simplemente prediciendo siempre esa clase.

* El **recall** (o sensibilidad) mide qué proporción de los casos positivos reales fue correctamente identificada por el modelo. Es especialmente importante cuando **no detectar un caso positivo tiene un costo alto**, como en diagnósticos médicos o detección de fraudes.

* El **precision** indica qué proporción de las predicciones positivas del modelo fueron realmente correctas. Esta métrica es clave cuando **los falsos positivos son costosos**.

* El **F1-score** combina precision y recall en una sola métrica, calculando su media armónica. Es útil cuando se busca un equilibrio entre ambas y cuando las clases están desbalanceadas. Un F1-score alto indica que el modelo logra un buen compromiso entre detectar casos positivos y no generar demasiados errores.

---

### Matriz de confusión

La **matriz de confusión** es una herramienta fundamental para entender en detalle el comportamiento de un modelo de clasificación. Muestra cuántas predicciones fueron correctas e incorrectas, separando los resultados en:

* verdaderos positivos,
* verdaderos negativos,
* falsos positivos,
* falsos negativos.

A partir de esta matriz se calculan métricas como accuracy, precision, recall y F1-score. En scikit-learn, la matriz de confusión se obtiene con la función `confusion_matrix()`, mientras que el resumen completo de métricas se genera con `classification_report()`, que presenta precision, recall y F1-score para cada clase de forma ordenada y legible.

Estas métricas permiten evaluar el modelo **desde múltiples perspectivas**, evitando conclusiones simplistas y ayudando a decidir si el modelo es adecuado para el problema que se está resolviendo.

## Carga del dataset

Vamos a utilizar el archivo `Titanic-Dataset.csv`, que ya deberías tener cargado en la carpeta de trabajo de Google Colab.

En el siguiente bloque de código vamos a importar las librerías necesarias y leer el archivo para comenzar a explorar los datos.


In [ ]:
# Importamos las librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# Configuramos el estilo de los gráficos
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (10, 5)

# Cargamos el archivo CSV del Titanic
ruta = "Titanic-Dataset.csv"
df = pd.read_csv(ruta)

# Mostramos las primeras filas para verificar la carga
df.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Preparación del dataset

Antes de entrenar el modelo, vamos a preparar los datos. El objetivo es predecir si una persona **sobrevivió** (`Survived`) o no.

Para eso, seleccionaremos algunas columnas que pueden ayudarnos a hacer esa predicción:
- `Pclass`: la clase del pasajero (1ª, 2ª o 3ª)
- `Sex`: género (vamos a codificarlo)
- `Age`: edad
- `SibSp`: cantidad de hermanos/cónyuges a bordo
- `Parch`: cantidad de padres/hijos a bordo

Como `Sex` es una variable de texto, necesitamos convertirla a números. Además, verificaremos que no haya valores faltantes en las columnas seleccionadas. Por último, separaremos las variables predictoras (`X`) y la variable objetivo (`y`), que en este caso es `Survived`.


In [ ]:
# Creamos una copia del DataFrame original para trabajar
df_modelo = df.copy()

# Codificamos la variable 'Sex': male = 0, female = 1
df_modelo['Sex_encoded'] = df_modelo['Sex'].map({'male': 0, 'female': 1})

# Seleccionamos las columnas a utilizar
columnas = ['Survived', 'Pclass', 'Sex_encoded', 'Age', 'SibSp', 'Parch']
df_modelo = df_modelo[columnas]

# Eliminamos filas con valores faltantes en las columnas seleccionadas
df_modelo = df_modelo.dropna()

# Definimos las variables predictoras (X) y la variable objetivo (y)
X = df_modelo[['Pclass', 'Sex_encoded', 'Age', 'SibSp', 'Parch']]
y = df_modelo['Survived']

# Verificamos que no haya valores nulos
print("¿Quedan valores nulos?", df_modelo.isna().sum().sum() > 0)

# Mostramos las primeras filas de las variables seleccionadas
X.head()


¿Quedan valores nulos? False


,Pclass,Sex_encoded,Age,SibSp,Parch
0,3,0,22.0,1,0
1,1,1,38.0,1,0
2,3,1,26.0,0,0
3,1,1,35.0,1,0
4,3,0,35.0,0,0


## División del dataset y entrenamiento del modelo

Antes de entrenar el modelo, vamos a dividir el dataset en dos partes:

- **Conjunto de entrenamiento** (80%): lo usaremos para ajustar el modelo.
- **Conjunto de prueba** (20%): lo usaremos para ver cómo se comporta el modelo con datos que no vio durante el entrenamiento.

Esto es importante porque queremos saber **si el modelo generaliza bien** y no solo memoriza los datos.

Una vez divididos los datos, vamos a entrenar un modelo de **regresión logística**. Este modelo estima la probabilidad de que una persona haya sobrevivido y luego decide, según un umbral (por defecto 0.5), si clasificarla como sobreviviente (`1`) o no (`0`).


In [ ]:
# Dividimos el dataset en entrenamiento (80%) y prueba (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Creamos el modelo de regresión logística
modelo = LogisticRegression()

# Entrenamos el modelo con los datos de entrenamiento
modelo.fit(X_train, y_train)

# Mostramos los coeficientes aprendidos por el modelo
coeficientes = pd.DataFrame({
    'Variable': X.columns,
    'Coeficiente': modelo.coef_[0]
})

print("Intercepto del modelo:", modelo.intercept_[0])
coeficientes


Intercepto del modelo: 3.0447177968459647


,Variable,Coeficiente
0,Pclass,-1.280119
1,Sex_encoded,2.611885
2,Age,-0.048807
3,SibSp,-0.338201
4,Parch,-0.031116


## Evaluación del modelo

Una vez entrenado el modelo, es fundamental evaluar **qué tan bien clasifica** los casos en el conjunto de prueba.

Vamos a utilizar dos herramientas principales:

- `classification_report`, que muestra **precisión**, **recall** y **F1-score** para cada clase, además de un promedio general.
- `confusion_matrix`, que muestra una tabla con los aciertos y errores del modelo, separando los verdaderos positivos, falsos positivos, falsos negativos y verdaderos negativos.

Como vimos, estas métricas nos permiten no solo ver cuántas veces acierta el modelo, sino también **cómo se equivoca**, lo cual es clave en problemas de clasificación binaria.


In [ ]:
# Generamos predicciones sobre el conjunto de prueba
y_pred = modelo.predict(X_test)

# Mostramos el reporte de métricas
print("Reporte de clasificación:\n")
print(classification_report(y_test, y_pred))

# Mostramos la matriz de confusión
print("Matriz de confusión:\n")
print(confusion_matrix(y_test, y_pred))


Reporte de clasificación:

              precision    recall  f1-score   support

           0       0.80      0.82      0.81        87
           1       0.70      0.68      0.69        56

    accuracy                           0.76       143
   macro avg       0.75      0.75      0.75       143
weighted avg       0.76      0.76      0.76       143

Matriz de confusión:

[[71 16]
 [18 38]]


## Interpretación de los resultados

El modelo de regresión logística alcanzó una **precisión general (accuracy)** del **76%**, lo que significa que acertó en aproximadamente 3 de cada 4 predicciones en el conjunto de prueba. Pero más allá del porcentaje total de aciertos, las **métricas por clase** nos permiten entender mejor su comportamiento.

Para la clase `0` (personas que **no sobrevivieron**), el modelo tiene una **precisión de 0.80** y un **recall de 0.82**. Esto significa que acierta la mayoría de las veces cuando predice que alguien no sobrevivió, y además logra capturar correctamente la mayor parte de los verdaderos casos negativos.

En cambio, para la clase `1` (personas que **sí sobrevivieron**), el modelo tiene una **precisión de 0.70** y un **recall de 0.68**. Es decir, se equivoca más seguido al predecir supervivientes, y también **deja escapar** una cantidad significativa de casos reales de personas que sí sobrevivieron.

La **matriz de confusión** ayuda a visualizar esto:  
- El modelo predijo correctamente **71 casos negativos** y **38 casos positivos**.  
- Cometió **16 falsos positivos** (personas que predijo como sobrevivientes pero no lo eran).  
- Y **18 falsos negativos** (personas que realmente sobrevivieron, pero que fueron clasificadas como no sobrevivientes).

En conjunto, estos resultados indican que el modelo tiene un **comportamiento equilibrado**, pero todavía tiene margen de mejora, especialmente en su capacidad de **detectar correctamente los casos positivos** (supervivientes). Esto podría ser importante en un contexto donde **detectar los positivos tiene mayor valor** o consecuencias prácticas.

Recordá que estas métricas no solo sirven para evaluar, sino para **tomar decisiones informadas**: si el problema fuera de seguridad, salud o justicia, elegir la métrica adecuada sería crucial.


## Conclusión y cierre

En este cuaderno entrenamos y evaluamos nuestro primer modelo de **clasificación binaria** utilizando **regresión logística**. A partir de las características personales de los pasajeros del Titanic, buscamos predecir si sobrevivieron o no.

Los resultados muestran un modelo razonablemente preciso, pero también evidencian que **acierta más en predecir quién no sobrevivió que en identificar correctamente a los que sí lo hicieron**. Este desequilibrio no es inusual, y pone en evidencia la importancia de usar métricas específicas como **precisión, recall y F1-score**, en lugar de quedarnos con un único número.

No está mal para ser la primera versión del modelo, pero todavía queda trabajo por hacer. Quizás otra técnica de clasificación pueda mejorar los resultados.

En el próximo cuaderno vamos a explorar otro enfoque: el algoritmo de los **K vecinos más cercanos (KNN)**. En lugar de ajustar una fórmula matemática, este modelo clasifica los casos **buscando similitudes** entre las observaciones. Es una forma distinta de pensar el problema, que puede resultar muy potente cuando las distancias importan más que las ecuaciones.

Seguimos avanzando, paso a paso, en el mundo del aprendizaje supervisado.
